# Introduction

So in the last notebook, we imported the data and then took a high level look at the data. For this notebook, we are going to jump into the data, exploring it a little bit more and then cleaning it by handling incorrect data and missing data, both of which will be explained in detail later. The ultimate goal of this notebook is to get our data to a place where it is ready to be used to analyzed and used to derive some insight.

# Importing Libraries and Data

In [1]:
# Import Libraries

import pandas as pd

In [5]:
# Ingest Data

df = pd.read_csv(r'C:\Users\jpau\Documents\Projects\Airforce\data\flight_data.csv')

# Missing Data

So when working with data, often the datasets that you will receive will be not fit for use right away. This may be for a variety of reasons such a missing data or data that is in an unusable format or even data is that incorrect. So, generally, the first step in understanding the data is to explore and handle these unclean and missing portions of data. 

To do this we are going to see what data is missing. To do this we are going to use the isnull() method and the sum() method. In effect, the isnull() method tells you whether or not an element is null (doesn't contain any information) and the sum() method will add up all of these nulls. 

In [28]:
# Missing Info

df.isnull().sum()

airport                     0
track                       0
stid                        0
call_sign                   0
time                        0
event                  193709
status                      0
departure_airport      329940
destination_airport    329940
timestamp                   0
dtype: int64

So there seems to be 3 columns with nulls in them, event, departure_airport, and destination_airport. Generally, there are 2 ways in which you can go about working with missing data, either you can impute or guessimate them based off prior knowledge or some statistical technique or you can remove them. 

For the departure and destination airports, in general incomplete flights are not all that useful, so we will remove those. In general, this data is derived form the SPDPS flight plan data, therefore there might be a problem that we cannot control. So we will drop observations with these missing values. 

Before we drop these null values, let us actually take a look at these observations. To do this will we need to subset the data with only those that have a null value in the departure airport and destination airport. 

In [29]:
index = df['departure_airport'].isnull()

df[index].head()

,airport,track,stid,call_sign,time,event,status,departure_airport,destination_airport,timestamp
1,KPIT,3534,339554,SWA4052,2020-01-01 00:00:08,off,airborne,NaN,NaN,2020-01-01 00:00:00
2,KMSY,4057,68290,SWA5830,2020-01-01 00:00:08,on,onsurface,NaN,NaN,2020-01-01 00:00:00
13,KSFO,3636,3605541,UAL595,2020-01-01 00:00:43,on,onsurface,NaN,NaN,2020-01-01 00:00:00
30,KCLE,3430,3486145,UAL1530,2020-01-01 00:02:01,off,airborne,NaN,NaN,2020-01-01 00:00:00
47,KMCI,2491,3857372,SWA1376,2020-01-01 00:02:47,off,airborne,NaN,NaN,2020-01-01 00:00:00


So we see that indeed there is nothing we can really do to fix these airports so let's drop them. To do this, we first need to subset the data to grab all those that are not null, and then we can make a new dataset with them. In this case we will use a method that is very similar to the isnull() method, it is called the notnull() method. It does exactly the opposite of the isnull() method. Next, we will subset the data in order and assign it to a new dataframe. Why don't you try doing this?

In [30]:
# Exercise 

index = None
new_df = None


In [6]:
# Answer

index = df['departure_airport'].notnull()
new_df = df[index].copy()

Nice! Let us do a check to see if the null values in the departure airport column were actually removed.

In [32]:
# Exercise

answer = None
print("Columns and their types:\n", answer)

Columns and their types:
 None


In [7]:
# Answer

answer = new_df.isnull().sum()
print("Columns and their types:\n", answer)

Columns and their types:
 airport                    0
call_sign                  0
time                       0
event                  26717
departure_airport          0
destination_airport        0
dtype: int64


One thing to notice is that when we took care of all the missing departure airports, we also took care of all the destination airports. Nice! So let's now move onto the missing events. 

First thing with the events column is that we are actually going to take a look at what kinds of events there are. We can do this by subsetting the data and then applying the unique() method to it. 

In [8]:
new_df['event'].unique()

array(['on', 'off', nan], dtype=object)

You'll notice that there are only 2 events, Off and On. These represent take offs and landings respectively.

Given that there are only two unique values and based off the reality that we can actually fill these in based upon the airport column and departure/destination, instead of dropping these columns, let's actually try to impute them. So to clarify, if the airport column matches the departure column, then we know the event must be a take off and if it matches the destination column, then the event must be a landing. Thus, we can assign values based on these things.

In [9]:
on_index = (new_df['event'].isnull()) & (new_df['airport'] == new_df['departure_airport'])
new_df.loc[on_index,'event'] = 'Off'

In the code above, we provide the two conditions for the missing events that we are looking for. They must first be null and secondly, they must be a take off. If these two conditions are met, then we can assign them to a value of 'Off'. You'll notice that there is a &. This is an bitwise operator that indicates that both conditions need to be true in order to return a true value. The usage of .loc in subsetting is just another way to subset the data. It is primarily used for label based indexing (eg. usage of the column 'events). Why don't you try to impute the values of the takeoffs. 

In [36]:
# Exercise

off_index = None


In [10]:
# Answer

off_index = (new_df['event'].isnull()) & (new_df['airport'] == new_df['destination_airport'])
new_df.loc[off_index,'event'] = 'On'

Now that that is done. Let us check to see if we have any last missing values. 

In [38]:
new_df.isnull().sum()

airport                  0
track                    0
stid                     0
call_sign                0
time                     0
event                  118
status                   0
departure_airport        0
destination_airport      0
timestamp                0
dtype: int64

Well, there seems to be a few more missing values. Let's take a look at these rows. 

In [39]:
# Exercise

index = None
new_df[index].head()

In [11]:
# Answer

index = new_df['event'].isnull() 
new_df[index].head()

,airport,call_sign,time,event,departure_airport,destination_airport
157970,KMDW,SWA1975,2020-07-30 17:53:35,NaN,KMCO,KPHL
221334,KSNA,SWA1908,2020-03-11 01:41:16,NaN,KPHX,KSEA
313461,KBWI,SWA28,2020-01-28 02:02:10,NaN,KHOU,KDAL
331997,KLAX,AAL2597,2020-02-10 05:28:09,NaN,KDFW,KBUR
338196,KSTL,AAL2408,2020-07-19 17:54:10,NaN,KCLT,KORD


It seems like neither the departure airport nor the destination airport matches the airport column. This is weird and because we don't know whether it is a take off or landing. Let us remove these observations.

In [41]:
# Exercise
index = None
complete_df = None

In [12]:
# Answer
index = new_df['event'].notnull()
complete_df = new_df[index]

Let's do one last check to see if we are finally done.

In [43]:
complete_df.isnull().sum()

airport                0
track                  0
stid                   0
call_sign              0
time                   0
event                  0
status                 0
departure_airport      0
destination_airport    0
timestamp              0
dtype: int64

Hurray! We now have a complete dataset to work with. Before we can move on to analyzing it, we need to see if all of the data in the dataset is correct. We will do this in the next section